Only run if in Colab!

In [ ]:
!git clone https://github.com/ratinac-nikola/rbm.git
%cd rbm
%pip install -r requirements.txt

In [1]:
%pwd

'/home/nikola/projects/master-rad/rbm'

### Initialization

In [1]:
from sklearn.model_selection import train_test_split
import torch
import config

if torch.cuda.is_available():
    print("Using GPU")
    device = torch.device('cuda')
else:
    print("Using CPU")
    device = "cpu"

Using CPU


In [2]:
from dataset import Dataset
dataset = Dataset(device=device)
tensorData = dataset.getDatasetAsTensor()
tensorData = tensorData[tensorData.sum(dim=[1,2]) != 0]
trainset, testset = train_test_split(tensorData, train_size=0.8)
trainset.shape, testset.shape

(torch.Size([216, 3706, 5]), torch.Size([54, 3706, 5]))

In [4]:
from rbm import RBM
rbm = RBM(trainset.shape[1], n_hidden=10, device=device)

RBM created   qweq.


In [5]:
for epoch in range(1, config.epochs + 1):
    i = 1
    for case in trainset:

        goodSample = case
        badSample = case

        for k in range(10):
            _, hk = rbm.sample_h(badSample)  # forward pass
            _, badSample = rbm.sample_v(hk)  # backward pass

        rbm.train(goodSample, badSample)
        i += 1
        #if i % 10 == 0:
            #print(f"Case {i} done")
    print(f"Epoch {epoch} done.", end=" ")
    rmse = 0
    for case in testset:
        reconstruction = rbm.reconstruct(case)
        reconstruction = reconstruction[case.sum(dim=1) > 0]
        case = case[case.sum(dim=1) > 0]
        case = torch.argmax(case, dim=1) + 1
        reconstruction = torch.argmax(reconstruction, dim=1) + 1
        
        #print(f"Case:{case}")
        #print(f"Reco:{reconstruction}")
        #print(f"C-R :{case-reconstruction}")
        rmse += torch.sqrt(torch.sum((case - reconstruction)**2)/len(case))
    print(rmse/len(testset))


Epoch 1 done. tensor(1.7961)
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/nikola/projects/master-rad/rbm/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_636707/627875862.py", line 9, in <cell line: 1>
    _, hk = rbm.sample_h(badSample)  # forward pass
  File "/home/nikola/projects/master-rad/rbm/rbm.py", line 35, in sample_h
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/nikola/projects/master-rad/rbm/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 1982, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/nikola/projects/master-rad/rbm/env/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/home/nikola/projects/master-rad/rbm/env/lib/python3.8/site-packages/IPyt